# データベースに平衡計算の結果を登録

アクセス可能なデータベースを準備する必要があります。
計算結果を保存する場所を用意する必要があります。  
その上で、ルートディレクトリにある.env_exampleを参考に同じ場所に.envを作成して、使用するデータベース名やユーザー名などを記述してください。

In [ ]:
# Setting
table_name='test_bin'
comment = "check range of aspect_ratio"
# 計算結果を保存するディレクトリ
dir_store = '\\\\quest-db.riam.kyushu-u.ac.jp\\share\\equilibrium'

In [ ]:
import os
os.chdir('../..')
import itertools
import numpy as np
import copy
import sub.plot as pl
import sub.functions as sb
import sub.db as sdb
from global_variables import gparam
gl = gparam()

In [ ]:
# ディレクトリの有無確認
dir = os.path.join(dir_store, table_name)
if not os.path.exists(dir):
    os.makedirs(dir)
    
# データベース設定
dbeq = sdb.DB_equilibrium()
dbeq.set_table(table_name, comment=comment)
chdat = sdb.DB(table_name)

In [ ]:
# define fundamental condition
condition = {
    # TF current
    'cur_tf':{'tf': +50.0e+3, 'turn': 16},
    
    # initial plasma profile
    'cur_ip':{'ip':+100.0e+3, 'r0':0.65, 'z0':0.00, 'radius':0.3, 'degree': 2.0},
    
    # PF currents
    'cur_pf':{'hcult16':0.0,'pf17t12':-2.0e+3, 'pf26t36':-2.0e+3,'pf4_1ab3_cc2':-1.0e+3, 'pf35_1': 4.0e+3, 'pf35_2': 4.0e+3, },
        
    # number of coefficients
    'num_dpr':1, # dp/df
    'num_di2':1, # di2/df

    # flag to fix magnetic axis at initial plasma profile (r0, z0) 
    #'fix_pos': True,

    # calculate flux (r, z): result is set to 'fl_val'.
    # 'fl_pos':{'flc8':(0.1985, 0.450), 'f_im':(0.1985, 0.0), 'flc17':(0.1985, -0.450),
    #           'fls1':(1.374, 0.450), 'fls5':(1.374, 0.0), 'fls9':(1.374, -0.481)},
    # calculate Br(r, z): result is set to 'br_val'
    #'br_pos':{'r1000z0':(1.0, 1.0)},
    # calculate Bz(r, z): result is set to 'bz_val'
    'bz_pos':{'r0zp800':(0.0, 0.8), 'r0zp400':(0.0, 0.4), 'r0z0':(0.0, 0.0), 'r0zm400':(0.0, -0.4), 'r0zm800':(0.0, -0.8),
              'r1450zp400':(1.45, 0.4), 'r1450z0':(1.45, 0.0), 'r1450zm400':(1.45, -0.4), },
    }

In [ ]:
# define parameters to change

# linspaceで値を指定すると、後から計算領域を広げる際に煩わしくなる。
# 従って、具体的に値を指定していった方が良いかもしれない。

# r0 = np.linspace(0.5, 0.7, 3)
# pf17 = np.linspace(-5.0e+3, 0.0e+3, 5)
# pf26 = np.linspace(-5.0e+3, 0.0e+3, 5)
# pf4  = np.linspace(-5.0e+3, 0.0e+3, 5)
# pf351 = np.linspace(0, 5.0e+3, 5)
# pf352 = np.linspace(0, 5.0e+3, 5)

r0 = [0.5, 0.6, 0.7]
pf17 = [-5.0, -4.0, -3.0, -2.0, -1.0, 0.0]
pf26 = [-5.0, -4.0, -3.0, -2.0, -1.0, 0.0]
pf4 = [-5.0, -4.0, -3.0, -2.0, -1.0, 0.0]
pf351 = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
pf352 = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]

r0 = [0.6, ]
pf17 = [-1.0, ]
pf26 = [-1.0, ]
pf4 = [0.0]
pf351 = [0.0]
pf352 = [0.0]

pf17  = np.array(pf17) * 1.0e+3
pf26  = np.array(pf26) * 1.0e+3
pf4   = np.array(pf4) * 1.0e+3
pf351 = np.array(pf351) * 1.0e+3
pf352 = np.array(pf352) * 1.0e+3

In [ ]:
# Calculate equilibrium and regist the data
for v0, v1, v2, v3, v4, v5 in list(itertools.product(r0, pf17, pf26, pf4, pf351, pf352)):
   print(v0, v1, v2, v3, v4, v5)
   cnd = copy.deepcopy(condition)
   cnd['cur_ip']['r0']         = v0
   cnd['cur_pf']['pf17t12']    = v1
   cnd['cur_pf']['pf26t36']    = v2
   cnd['cur_pf']['pf4_1ab3_cc2'] = v3
   cnd['cur_pf']['pf35_1']     = v4
   cnd['cur_pf']['pf35_2']     = v5
   
   # 既にdbにあるかチェック
   # データの有無を確認しても良いが、データ数が多くなってくると、その確認にも時間がかかるようになってくる。
   # 従って、最初から重複しないようにパラメータを設定するのが一番良い。
   # sql文でLIMIT 1は重要。これがないと全ての範囲を検索するので時間がかかる。
   
   #sql = 'cur_ip_r0 = %s AND cur_pf_pf17t12=%s AND cur_pf_pf26t36=%s AND cur_pf_pf4_1ab3_cc2=%s AND cur_pf_pf35_1=%s AND cur_pf_pf35_2=%s LIMIT 1'
   #ch = chdat.select_df(['id'], sql, [float(e) for e in [v0, v1, v2, v3, v4, v5]])

   # sql = 'cur_ip_r0 > %s AND cur_ip_r0 < %s' # r0はイコールが使えず、範囲指定する必要があった。
   # val = [float(v0)*0.99, float(v0)*1.01]
   # sql += 'AND cur_pf_pf17t12=%s AND cur_pf_pf26t36=%s AND cur_pf_pf4_1ab3_cc2=%s AND cur_pf_pf35_1=%s AND cur_pf_pf35_2=%s LIMIT 1'
   # val += [float(e) for e in [v1, v2, v3, v4, v5]]
   # ch = chdat.select_df(['id'], sql, val)
   
   # if len(ch) != 0:
   #    continue

   # 実際にデータベースへ登録をするときは、下を有効にする。   
   # cnd = sb.calc_equilibrium(cnd, verbose=0)
   # dat = sdb.disassemble_condition(cnd)
   # id = dbeq.get_next_id()
   # dbeq.add_data(dat)
   # fdir = os.path.join(dir, f"{id}.npy")
   # np.save(fdir, cnd)

In [ ]:
# to read condition
#cnd = np.load(fdir, , allow_pickle=True).item()